## 3 - Introductory Example - 1-Axis tracker by hour (gendaylit)

Example demonstrating the use of Radiance gendaylit for 1-axis tracking.

#### Types of 1-axis tracking simulations:

<b>CumulativeSky: False </b>. This uses Gendaylit function, which performs the simulation hour by hour. A good computer and a some patience are needed for doing the 4000 daylight-hours of the year (~1 day on a 32GB RAM, Windows 10 i7-8700 CPU @ 3.2GHz with 6 cores), or else a high-performance-computing for handling full year simulations. The procedure can be broken into shorter steps for one day or a single timestamp simulation which is exemplified below.


### Steps:
<ol>
    <li> <a href='#step1'> Create a folder for your simulation, and Load bifacial_radiance </a></li> 
    <li> <a href='#step2'> Create a Radiance Object </a></li> 
    <li> <a href='#step3'> Set the Albedo </a></li> 
    <li> <a href='#step4'> Download Weather Files </a></li>    
    <ul> (VERY SIMILAR TO FIXED TILT EXAMPLE UNTIL HERE) </ul> 
    <li> <a href='#step5'> Set Tracking Angles </a></li> 
    <li> <a href='#step6'> Generate the Sky </a></li> 
    <li> <a href='#step7'> Define a Module type </a></li> 
    <li> <a href='#step8'> Create the scene </a></li> 
    <li> <a href='#step9'> Combine Ground, Sky and Scene Objects </a></li> 
    <li> <a href='#step10'> Analyze and get results </a></li> 
    <li> <a href='#step11'> Clean Results </a></li>   
   
</ol>

And finally:  <ul> <a href='#condensed'> Condensed instructions </a></ul>   

<a id='step1'></a>


## 1. Create a folder for your simulation, and load bifacial_radiance 

First let's set the folder where the simulation will be saved. By default, this is the TEMP folder in the bifacial_radiance distribution.

The lines below find the location of the folder relative to this Jupyter Journal. You can alternatively point to an empty directory (it will open a load GUI Visual Interface) or specify any other directory in your computer, for example:

#### testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal3'


In [1]:
import os
testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


#### Pay attention: different importing method:

So far we've used "from bifacial_radiance import *" to import all the bifacial_radiance files into our working space in jupyter. For this journal we will do a "import bifacial_radiance" . This method of importing requires a different call for some functions as you'll see below. For example, instead of calling demo = RadianceObj(path = testfolder) as on Tutorial 2, in this case we will neeed to do demo = bifacial_radiance.RadianceObj(path = testfolder). 

In [2]:
import bifacial_radiance
import numpy as np
import pprint    # We will be pretty-printing the trackerdictionary throughout to show its structure.

<a id='step2'></a>

## 2. Define all your system variables

Just like in the condensed version show at the end of Tutorial 2, for this tutorial we will be starting all of our system variables from the beginning of the jupyter journal, instead than throughout the different cells (for the most part)

In [18]:
moduletype = 'Custom Cell-Level Module'    # We will define the parameters for this below in Step 4.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2'
albedo = "litesoil"      # this is one of the options on ground.rad
lat = 37.5   
lon = -77.6

# Scene variables
nMods = 20
nRows = 7
hub_height = 2.3 # meters
pitch = 10 # meters      # We will be using pitch instead of GCR for this example.

# Traking parameters
cumulativesky = False
limit_angle = 45 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

#makeModule parameters
# x and y will be defined later on Step 4 for this tutorial!!
xgap = 0.01
ygap = 0.10
zgap = 0.05
numpanels = 2
torquetube = True
axisofrotationTorqueTube = False
diameter = 0.1
tubetype = 'Oct'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube of this material (0% reflectivity)

# Analysis parameters
modWanted = 9
rowWanted = 2
sensorsy = 6



<a id='step3'></a>

## 3. Create Radiance Object, Set Albedo and Weather

Same steps as previous two tutorials, so condensing it into one step. You hopefully have this down by now! :)


<div class="alert alert-warning">
Notice that we are doing bifacial_radiance.RadianceObj because we change the import method for this example!
</div>

In [19]:
demo = bifacial_radiance.RadianceObj(path = testfolder) 
demo.setGround(albedo) 
epwfile = demo.getEPW(lat = lat, lon = lon) 
metdata = demo.readWeatherFile(weatherFile = epwfile) 


path = C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!


<a id='step4'></a>

## 4. Make Module: Cell Level Module Example

Instead of doing a opaque, flat single-surface module, in this tutorial we will create a module made up by cells. We can define variuos parameters to make a cell-level module, such as cell size and spacing between cells. To do this, we will pass a dicitonary with the needed parameters to makeModule, as shown below.

<div class="alert alert-warning">
Since we are passing a cell-level dictionary, the values for module's x and y of the module will be calculated by the software -- no need to pass them (and if you do, they'll just get ignored)
    </div>

In [20]:
numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

mymodule = demo.makeModule(name=moduletype, torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=material, 
                xgap=xgap, ygap=ygap, zgap=zgap, numpanels=numpanels, 
                cellLevelModuleParams=cellLevelModuleParams, 
                axisofrotationTorqueTube=axisofrotationTorqueTube)


Module Name: Custom_Cell-Level_Module
REWRITING pre-existing module file. 
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module Custom Cell-Level Module successfully created


<a id='step5'></a>

## 5. Calculate GCR

In this example we passed the parameter "pitch". Pitch is the spacing between rows (for example, between hub-posts) in a field.
To calculate Ground Coverage Ratio (GCR), we must relate the pitch to the collector-width by:
    
![GCR = CW / pitch](../images_wiki/Journal3Pics/Equation_GCR.png)

The collector width for our system must consider the number of panels and the y-gap:
    
![CW](../images_wiki/Journal3Pics/Equation_CW.png)
    
Collector Width gets saved in your module parameters (and later on your scene and trackerdict) as "sceney". You can calculate your collector width with the equation, or you can use this method to know your GCR:

In [21]:
# For more options on makemodule, see the help description of the function.  
CW = mymodule['sceney']
gcr = CW / pitch
print ("The GCR is :", gcr)

The GCR is : 0.4284


<a id='step6'></a>

## 6. Set Tracking Angles

This function will read the weather file, and based on the sun position it will calculate the angle the tracker should be at for each hour. It will create metdata files for each of the tracker angles considered.

For doing hourly simulations, remember to set cumulativesky = False here!

In [7]:
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)

C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:449: RuntimeWarning: invalid value encountered in arccos
  wc = np.degrees(np.arccos(temp))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:473: RuntimeWarning: invalid value encountered in arccos
  aoi = np.degrees(np.arccos(np.abs(np.sum(sun_vec*panel_norm, axis=0))))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:568: RuntimeWarning: invalid value encountered in remainder
  surface_azimuth = surface_azimuth % 360
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:572: RuntimeWarning: invalid value encountered in arccos
  surface_tilt = 90 - np.degrees(np.arccos(dotproduct))


In [8]:
print ("Full trackerdict for all the year created by set1axis: %s " % (len(demo.trackerdict))) 


Full trackerdict for all the year created by set1axis: 4245 


set1axis initializes the trackerdictionary Trackerdict. Trackerdict contains all hours in the year as keys. For example: trackerdict['12_16_08']. It is a return variable on many of the 1axis functions, but it is also stored inside of your Radiance Obj (i.e. demo.trackerdict). In this journal we are storing it as a variable to mute the option (otherwise it prints the returned trackerdict contents every time)


In [9]:
pprint.pprint(trackerdict['12_16_08'])


{'dhi': 18, 'ghi': 18, 'surf_azm': 90.0, 'surf_tilt': 13.22, 'theta': -13.22}


In [10]:
pprint.pprint(demo.trackerdict['12_16_08'])

{'dhi': 18, 'ghi': 18, 'surf_azm': 90.0, 'surf_tilt': 13.22, 'theta': -13.22}


All of the following functions add up elements to trackerdictionary to keep track (ba-dum tupzz) of the Scene and simulation parameters. In advanced journals we will explore the inner structure of trackerdict. For now, just now it exists :)

<a id='step7'></a>

## 7. Generate the Sky


We will create skies for each hour we want to model with the function gendaylit1axis. 

If you don't specify a startdate and enddate, all the year will be created, which will take more time. 

For this example we are doing just two days in January. Format has to be a 'MM_DD' or 'MM/DD'

In [11]:
startdate = '01/13'     
enddate = '01/14'
trackerdict = demo.gendaylit1axis(startdate=startdate, enddate=enddate) 

Creating ~4000 skyfiles.  Takes 1-2 minutes
Created 20 skyfiles in /skies/


Since we passed startdate and enddate to gendaylit, it will prune our trackerdict to only the desired days.
Let's explore our trackerdict:

In [12]:
print ("\nTrimmed trackerdict by gendaylit1axis to start and enddate length: %s " % (len(trackerdict)))
print ("")
trackerkeys = sorted(trackerdict.keys())
print ("Option of hours are: ", trackerkeys)
print ("")
print ("Contents of trackerdict for sample hour:")
pprint.pprint(trackerdict[trackerkeys[0]])


Trimmed trackerdict by gendaylit1axis to start and enddate length: 20 

Option of hours are:  ['01_13_08', '01_13_09', '01_13_10', '01_13_11', '01_13_12', '01_13_13', '01_13_14', '01_13_15', '01_13_16', '01_13_17', '01_14_08', '01_14_09', '01_14_10', '01_14_11', '01_14_12', '01_14_13', '01_14_14', '01_14_15', '01_14_16', '01_14_17']

Contents of trackerdict for sample hour:
{'dhi': 15,
 'ghi': 15,
 'skyfile': 'skies\\sky2_37.5_-77.33_01_13_08.rad',
 'surf_azm': 90.0,
 'surf_tilt': 10.34,
 'theta': -10.34}


<a id='step8'></a>

## 8. Make Scene 1axis

We can use gcr or pitch fo our scene dictionary.

In [13]:
# making the different scenes for the 1-axis tracking for the dates in trackerdict2.

sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows}  

trackerdict = demo.makeScene1axis(trackerdict = trackerdict, moduletype = moduletype, sceneDict = sceneDict) 


Making ~20 .rad files for gendaylit 1-axis workflow (this takes a minute..)
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDict2
5
pitch 2 5
gcr  0.8568
Used the first sceneDic

In [14]:
pprint.pprint(trackerdict[trackerkeys[0]])

{'clearance_height': 1.9155344488542871,
 'dhi': 15,
 'ghi': 15,
 'radfile': 'objects\\1axis01_13_08_1.915_5_10.34_20x7_origin0,0.rad',
 'scene': <bifacial_radiance.main.SceneObj object at 0x000001D89CD4D128>,
 'skyfile': 'skies\\sky2_37.5_-77.33_01_13_08.rad',
 'surf_azm': 90.0,
 'surf_tilt': 10.34,
 'theta': -10.34}


The scene parameteres are now stored in the trackerdict. To view them and to access them:
    

In [16]:
print ("pitch =", pitch)
print ("CW = ", mymodule['sceney'])
print ("gcr = ", gcr)
print ("Note: Passed gcr to the sceneDict. Printing trackerdict after makeScene1axis")
pprint.pprint(demo.trackerdict[trackerkeys[5]]['scene'].__dict__)

pitch = 5
CW =  4.284
gcr =  0.2092
Note: Passed gcr to the sceneDict. Printing trackerdict after makeScene1axis
{'bifi': 1,
 'gcr': 0.8568,
 'moduleDict': {'bifi': 1,
                'cellModule': {'numcellsx': 6,
                               'numcellsy': 12,
                               'xcell': 0.156,
                               'xcellgap': 0.02,
                               'ycell': 0.156,
                               'ycellgap': 0.02},
                'modulefile': 'objects\\Custom_Cell-Level_Module.rad',
                'numpanels': 2,
                'offsetfromaxis': 0,
                'scenex': 1.046,
                'sceney': 4.284,
                'scenez': 0.1,
                'text': '! genbox black cellPVmodule 0.156 0.156 0.02 | xform '
                        '-t -0.44 -2.142 0 -a 6 -t 0.176 0 0 -a 12 -t 0 0.176 '
                        '0 -a 2 -t 0 2.192 0\r\n'
                        '! genbox black octtube1a 1.046 0.04142135623730951 '
                   

In [ ]:
pprint.pprint(demo.trackerdict[trackerkeys[0]]['scene'].scene.__dict__)

In [ ]:
print ("\nTrimmed trackerdict by gendaylit1axis to start and enddate length: %s " % (len(trackerdict)))
print ("")
print ("Contents of trackerdict for sample hour: %s " % ( trackerdict[trackerkeys[0]]))


#### Run one single index (super fast example):

In [ ]:
# Now this is the part that takes a long time, and will probably require parallel computing for doing more time points or the full year. 
# For this example we just run one hourly point:

demo2.makeOct1axis(trackerdict2,singleindex='01_01_11')

# This is for exemplifying the changes undergone in the trackerdict by each step. Just printing information.
print ("\n Contents of trackerdict for sample hour after makeOct1axis: \n trackerdict2['01_01_11']: \n %s \n" % ( trackerdict2['01_01_11']))

demo2.analysis1axis(trackerdict2,singleindex='01_01_11')

# This is for exemplifying the changes undergone in the trackerdict by each step. Just printing information.
print ("\n Contents of trackerdict for sample hour after makeOct1axis: \n trackerdict2['01_01_11']: \n %s \n" % ( trackerdict2['01_01_11']))

# Printing the results.
print('\n\n1-axis tracking hourly bifi gain: {:0.3}'.format(sum(demo2.Wm2Back) / sum(demo2.Wm2Front)))



#### Run a range of indexes: (not as fast as a single index, not as slow as all!)


In [ ]:
for time in ['01_01_11','01_01_12']:  # just two timepoints
    demo2.makeOct1axis(trackerdict2,singleindex=time)
    demo2.analysis1axis(trackerdict2,singleindex=time)

print('1-axis tracking hourly bifi gain: {:0.3}'.format(sum(demo2.Wm2Back) / sum(demo2.Wm2Front)))

#### Run the full trackingdictionary...
(this might considerably more time, depending on the number of entries on the trackerdictionary! You've been warned) 


In [ ]:
demo2.makeOct1axis(trackerdict2,singleindex=time)
demo2.analysis1axis(trackerdict2,singleindex=time)
print('1-axis tracking hourly bifi gain: {:0.3}'.format(sum(demo2.Wm2Back) / sum(demo2.Wm2Front)))


### Gendaylit for the WHOLE Year
And because you asked: this is the summarized version to run with gendaylit the WHOLE year. 
#### This will take ~4 days on a really good computer. IF you're sure this is what you want, uncomment and run below :)

In [ ]:
'''
demo2 = RadianceObj('Gendaylit_AllYear_Tracking',testfolder)  
demo2.setGround(0.2) 
epwfile = demo2.getEPW(37.5,-77.6) #pull TMY data for any global lat/lon
metdata = demo2.readEPW(epwfile) # read in the weather data   
module_type = 'Prism Solar Bi60'
sceneDict = {'pitch':1.695 / 0.33,'height':2.35, 'nMods': 20, 'nRows': 7}  
trackerdict2 = demo2.set1axis(cumulativesky = False)  # this cumulativesky = False key is crucial to set up the hourly workflow
trackerdict2 = demo2.gendaylit1axis()  # optional parameters 'startdate', 'enddate' inputs = string 'MM/DD' or 'MM_DD' 
trackerdict2 = demo2.makeScene1axis(trackerdict2, module_type,sceneDict, cumulativesky = False) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo2.makeOct1axis(trackerdict2)
demo2.analysis1axis(trackerdict2)
'''